In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
file_name = "flex.kr83m.1.s3mmp15mm.nexus.h5"
this_dir = "/Users/taylorcontreras/Development/Research/Odyssey_sipmstudy/"

In [ ]:
sns_response = pd.read_hdf(this_dir+file_name, 'MC/sns_response')

In [ ]:
# Sort to sum up all charges for each sipms
sns_response_sorted = sns_response.sort_values(by=['sensor_id'])
sipm_response = sns_response_sorted.loc[sns_response_sorted["sensor_id"] >999]

In [ ]:
top_sipms = sipm_response.groupby('event_id',as_index=False).apply(lambda grp: grp.nlargest(10,'charge'))

In [ ]:
top_sipms

In [ ]:
top_sipms_by_event = top_sipms.groupby('event_id')
top_10_sipms = [top_sipms_by_event.apply(lambda grp: grp.iloc[i]) for i in range(0,10)]

In [ ]:
for event, group in top_sipms_by_event:
    display(group.loc[:,'sensor_id':'charge'])
    

In [ ]:
[plt.hist(top_10_sipms[i]['charge'], alpha=0.5, label=str(i+1),bins=20,range=(0,250)) for i in range(0,5)]
plt.xlabel('Charge [pes]')
plt.ylabel('Top i-th sipm in charge per event')
plt.legend()

In [ ]:
mcs = [{'files':[file_name], 'pitch':5, 'dir':''}]
for mc in mcs:
    print("Looping in mcs")
    top_sipms = pd.DataFrame()

    for file in mc["files"]:
        print("Looping files in mc")
        sns_response = pd.read_hdf(file, 'MC/sns_response')

        # Sort to sum up all charges for each sipms                                                                
        sns_response_sorted = sns_response.sort_values(by=['sensor_id'])
        sipm_response = sns_response_sorted.loc[sns_response_sorted["sensor_id"] >999]

        top_10_sipms = sipm_response.groupby('event_id',as_index=False).apply(lambda grp: grp.nlargest(10,'charge'))
        top_sipms_by_event = top_10_sipms.groupby('event_id')

    mc["top_sipms"] = [top_sipms_by_event.apply(lambda grp: grp.iloc[i]) for i in range(0,10)]


In [ ]:
outdir = ''
for mc in mcs:
    for i in range(0,10):
        plt.hist(mc['top_sipms'][i]['charge'], bins=50, range=(0,250))
        plt.xlabel('charge [pes]')
        plt.title('NEXT-100, 3mmx3mm SiPMs, '+str(mc['pitch'])+' pitch, '+str(i)+' largest sipm signal per event')
        plt.savefig(outdir+mc['dir']+"_"+str(i)+"largest_sipm.png")
        plt.close()